It's simple to buy any product with a click and have it delivered to your door. Online shopping has been rapidly evolving over the last few years, making our lives easier. But behind the scenes, e-commerce companies face a complex challenge that needs to be addressed.

Uncertainty plays a big role in how the supply chains plan and organize their operations to ensure that the products are delivered on time. These uncertainties can lead to challenges such as stockouts, delayed deliveries, and increased operational costs.

You work for the Sales & Operations Planning (S&OP) team at a multinational e-commerce company. They need your help to assist in planning for the upcoming end-of-the-year sales. They want to use your insights to plan for promotional opportunities and manage their inventory. This effort is to ensure they have the right products in stock when needed and ensure their customers are satisfied with the prompt delivery to their doorstep.


## The Data

You are provided with a sales dataset to use. A summary and preview are provided below.

# Online Retail.csv

| Column     | Description              |
|------------|--------------------------|
| `'InvoiceNo'` | A 6-digit number uniquely assigned to each transaction |
| `'StockCode'` | A 5-digit number uniquely assigned to each distinct product |
| `'Description'` | The product name |
| `'Quantity'` | The quantity of each product (item) per transaction |
| `'UnitPrice'` | Product price per unit |
| `'CustomerID'` | A 5-digit number uniquely assigned to each customer |
| `'Country'` | The name of the country where each customer resides |
| `'InvoiceDate'` | The day and time when each transaction was generated `"MM/DD/YYYY"` |
| `'Year'` | The year when each transaction was generated |
| `'Month'` | The month when each transaction was generated |
| `'Week'` | The week when each transaction was generated (`1`-`52`) |
| `'Day'` | The day of the month when each transaction was generated (`1`-`31`) |
| `'DayOfWeek'` | The day of the weeke when each transaction was generated <br>(`0` = Monday, `6` = Sunday) |

**Analyze the Online Retail.csv dataset and build a forecasting model to predict 'Quantity' of products sold.**

1. Split the data into two sets based on the splitting date, "2011-09-25". All data up to and including this date should be in the training set, while data after this date should be in the test set. Return a pandas DataFrame, pd_daily_train_data, containing, at least, the columns "Country", "StockCode", "InvoiceDate", "Quantity".

2. Using your test set, calculate the Mean Absolute Error (MAE) for your forecast model for the 'Quantity' sold? Return a double (float) named mae.

3. How many units are expected to be sold during the week 39 of 2011? Store as an integer variable called quantity_sold_w39.

# Tiền xử lý

In [93]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, sum, count, lit
import pandas as pd

In [94]:
spark = SparkSession.builder.appName("OnlineRetailAnalysis").getOrCreate()

In [95]:
file_path = "/Users/tiendat02/Documents/big-data/Final HW-Demand Forecasting/OnlineRetail.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [96]:
print(df.count())
print(len(df.columns))

541909
8


In [97]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [98]:
df.show(10)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [99]:
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|       1454|       0|          0|        0|    135080|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



Có khá nhiều ID khách hàng bị thiếu => Xoá 

In [100]:
df = df.dropna(subset=["Description", "CustomerID"])

In [101]:
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|          0|       0|          0|        0|         0|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [102]:
df_duplicates = df.groupBy(df.columns).count().filter("count > 1")
df_duplicates.show()

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|count|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+-----+
|   537051|    85175|CACTI T-LIGHT CAN...|      32| 12/5/2010 11:12|     0.42|     15708|United Kingdom|    2|
|   541985|    22588|CARD HOLDER GINGH...|       2| 1/24/2011 15:52|     2.55|     15052|United Kingdom|    2|
|   542192|    84378|SET OF 3 HEART CO...|       3| 1/26/2011 11:27|     1.25|     14049|United Kingdom|    2|
|   537208|   90082D|DIAMANTE BOW BROO...|       1| 12/5/2010 15:12|     6.35|     15889|United Kingdom|    2|
|   538368|    20973|12 PENCIL SMALL T...|       1|12/12/2010 10:57|     0.65|     15503|United Kingdom|    2|
|   538371|    21216|SET 3 RETROSPOT T...|       1|12/12/2010 11:07|     4.95|     15570|United Kingdom|    2|
|

Không có hàng nào trùng lặp vô lý => giữ nguyên


In [103]:
df.select("InvoiceDate").show(5, truncate=False)

+--------------+
|InvoiceDate   |
+--------------+
|12/1/2010 8:26|
|12/1/2010 8:26|
|12/1/2010 8:26|
|12/1/2010 8:26|
|12/1/2010 8:26|
+--------------+
only showing top 5 rows



In [104]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



Cột InvoiceDate đang có kiểu dữ liệu là "string" => đổi sang dạng "date"

In [105]:
df = df.withColumn("InvoiceDate", to_date("InvoiceDate", "M/d/yyyy H:mm"))

In [106]:
df.printSchema()
df.show()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6| 2010-12-01|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6| 2010-12-01|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8| 2010-12-01|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6| 2010-12-01| 

1, Split the data into two sets based on the splitting date, "2011-09-25". All data up to and including this date should be in the training set, while data after this date should be in the test set. Return a pandas DataFrame, pd_daily_train_data, containing, at least, the columns "Country", "StockCode", "InvoiceDate", "Quantity".

In [107]:
split_date = "2011-09-25"

In [108]:
train_df = df.filter(col("InvoiceDate") <= split_date)
test_df = df.filter(col("InvoiceDate") > split_date)

In [109]:
train_df = train_df.select("Country", "StockCode", "InvoiceDate", "Quantity")

In [110]:
pd_daily_train_data = train_df.toPandas()

In [111]:
test_df = test_df.select("Country", "StockCode", "InvoiceDate", "Quantity")
pd_daily_test_data = train_df.toPandas()

In [112]:
pd_daily_train_data.head()

,Country,StockCode,InvoiceDate,Quantity
0,United Kingdom,85123A,2010-12-01,6
1,United Kingdom,71053,2010-12-01,6
2,United Kingdom,84406B,2010-12-01,8
3,United Kingdom,84029G,2010-12-01,6
4,United Kingdom,84029E,2010-12-01,6


2, Using your test set, calculate the Mean Absolute Error (MAE) for your forecast model for the 'Quantity' sold? Return a double (float) named mae.

In [113]:
# loại bỏ các dòng có giá trị "Quantity" bị âm
pd_daily_train_data = pd_daily_train_data[pd_daily_train_data['Quantity'] >= 0]
pd_daily_test_data = pd_daily_test_data[pd_daily_test_data['Quantity'] >= 0]

In [114]:
pd_daily_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 257815 entries, 0 to 263981
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Country      257815 non-null  object
 1   StockCode    257815 non-null  object
 2   InvoiceDate  257815 non-null  object
 3   Quantity     257815 non-null  int32 
dtypes: int32(1), object(3)
memory usage: 8.9+ MB


In [115]:
from sklearn.preprocessing import LabelEncoder
# Mã hóa cột 'Country'
label_encoder = LabelEncoder()
pd_daily_train_data['Country_encoded'] = label_encoder.fit_transform(pd_daily_train_data['Country'])
pd_daily_test_data['Country_encoded'] = label_encoder.transform(pd_daily_test_data['Country'])

In [116]:
# Mã hóa cột 'StockCode'
stock_code_encoder = LabelEncoder()
pd_daily_train_data['StockCode_encoded'] = stock_code_encoder.fit_transform(pd_daily_train_data['StockCode'])
pd_daily_test_data['StockCode_encoded'] = stock_code_encoder.transform(pd_daily_test_data['StockCode'])

In [117]:
# Chuyển đổi cột 'InvoiceDate' sang định dạng datetime
pd_daily_train_data['InvoiceDate'] = pd.to_datetime(pd_daily_train_data['InvoiceDate'], format='%m/%d/%Y')
pd_daily_test_data['InvoiceDate'] = pd.to_datetime(pd_daily_test_data['InvoiceDate'], format='%m/%d/%Y')

In [118]:
X_train = pd_daily_train_data[['Country_encoded', 'StockCode_encoded']]
y_train = pd_daily_train_data['Quantity']

X_test = pd_daily_test_data[['Country_encoded', 'StockCode_encoded']]
y_test = pd_daily_test_data['Quantity']

In [119]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [120]:
model = LinearRegression()

In [121]:
model.fit(X_train, y_train)

LinearRegression()

In [122]:
y_pred = model.predict(X_test)

In [123]:
mae = mean_absolute_error(y_test, y_pred)
print("MAE =",mae)

MAE = 13.566248786624111


3, How many units are expected to be sold during the week 39 of 2011? Store as an integer variable called quantity_sold_w39.

In [124]:
filtered_data = pd_daily_test_data[(pd_daily_test_data['InvoiceDate'].dt.year == 2011) &
                                   (pd_daily_test_data['InvoiceDate'].dt.isocalendar().week == 39)]

In [125]:
# Tổng số lượng bán được
quantity_sold_w39 = filtered_data['Quantity'].sum()
# đảm bảo kết quả là số nguyên
quantity_sold_w39 = int(quantity_sold_w39)
print(f"Tổng số lượng đơn vị bán được trong các tuần 39 =",quantity_sold_w39)

Tổng số lượng đơn vị bán được trong các tuần 39 = 0


In [126]:
# Lọc dữ liệu cho tuần gần kề để kiểm tra
weeks_to_check = [38, 40]  
filtered_data_nearby = pd_daily_test_data[(pd_daily_test_data['InvoiceDate'].dt.year == 2011) &
                                          (pd_daily_test_data['InvoiceDate'].dt.isocalendar().week.isin(weeks_to_check))]

print(filtered_data_nearby.head())
print(f"Tổng số lượng đơn vị bán được trong các tuần gần kề: {filtered_data_nearby['Quantity'].sum()}")


               Country StockCode InvoiceDate  Quantity  Country_encoded  \
252904          Cyprus    15056N  2011-09-19        33                7   
252905  United Kingdom     23293  2011-09-19         8               34   
252906  United Kingdom     23546  2011-09-19        25               34   
252907  United Kingdom     23549  2011-09-19        25               34   
252908  United Kingdom     23231  2011-09-19        25               34   

        StockCode_encoded  
252904                 19  
252905               2105  
252906               2260  
252907               2263  
252908               2055  
Tổng số lượng đơn vị bán được trong các tuần gần kề: 163656
